In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.plotsiii import plotiii1, plotiii2

In [2]:
# set seed
np.random.seed(120420)

### Tensorflow HMC 

In [3]:
import tensorflow as tf
print(tf.__file__)


/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages/tensorflow/__init__.py


In [4]:
import time
import tensorflow as tf
import tensorflow_probability as tfp

In [5]:
# set seed
np.random.seed(120420)

# target PDF function compatible with tensorflow
def target_pdf(x, alpha, beta):
    # prior varaibles
    a = -10
    b = 10
    c = 10
    likelihood = tfp.distributions.Cauchy(loc=alpha, scale=beta)
    prior = tfp.distributions.Uniform(low=[a,0], high=[b,c])
    return likelihood.prob(x) * prior.prob(x)

In [6]:
# hmc kernel
hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
  target_log_prob_fn=target_pdf,
  step_size=np.float64(.01),
  num_leapfrog_steps=200)

# run hmc
@tf.function
def run_chain(initial_state, num_results=30000, num_burnin_steps=10):
  return tfp.mcmc.sample_chain(
                            num_results=num_results,
                            num_burnin_steps=num_burnin_steps,
                            current_state=initial_state,
                            kernel=hmc_kernel,
                            trace_fn=lambda current_state, kernel_results: kernel_results)

num_chains, ndim = 8, 3
initial_state = np.random.normal(size=(num_chains, ndim))

t_start = time.time()
chains, kernel_results = run_chain(initial_state)
t_end = time.time()

TypeError: in user code:

    File "/var/folders/2g/9t7c77957pl9vb9x2t79vs2c0000gn/T/ipykernel_79316/1877366467.py", line 10, in run_chain  *
        return tfp.mcmc.sample_chain(
    File "/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/sample.py", line 333, in sample_chain  **
        previous_kernel_results = kernel.bootstrap_results(current_state)
    File "/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/hmc.py", line 565, in bootstrap_results
        kernel_results = self._impl.bootstrap_results(init_state)
    File "/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/metropolis_hastings.py", line 277, in bootstrap_results
        pkr = self.inner_kernel.bootstrap_results(init_state)
    File "/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/hmc.py", line 779, in bo

In [8]:
import tensorflow as tf
print(tf.__version__)


AttributeError: module 'tensorflow' has no attribute '__version__'

In [9]:
import importlib.util
spec = importlib.util.spec_from_file_location("tensorflow", "/Users/maxtalberg/miniconda3/envs/S2-env/lib/python3.8/site-packages")
tf = importlib.util.module_from_spec(spec)
spec.loader.exec_module(tf)
print(tf.__file__)


AttributeError: 'NoneType' object has no attribute 'loader'

In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_probability as tfp
print(tfp.__version__)


2.12.0
0.14.0
